Remark (6.5., 19:09): I am not yet done and will keep working on it

I will work on the routing algorithm here

# Dijkstra Algorithm

I will start with the Dijkstra Algorithm first. 

In [2]:
import geopandas as gpd
import numpy as np

I currently can't figure out how to import the "graph.py" file... So, as an easy workaround, I will just copy the file here: 

In [3]:
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.pos = (x, y)
        self.visited = False

# Graph implemented by adjacency list
class Graph:
    def __init__(self):
        self.adj = {}
        self.nodes = {}

    def add_edge(self, v, w, length):
        self.adj.setdefault(v, []).append((w, length))
        self.adj.setdefault(w, []).append((v, length))

    def add_node(self, node_idx, node):
        self.nodes[node_idx] = node

    def get_node(self, node_idx):
        return self.nodes[node_idx]

    def adj(self, v):
        return self.adj[v]

# Graph implemented by adjacency matrix
class Graph_matrix:
    def __init__(self):
        self.dis_matrix= np.zeros((4965, 4956))
        self.nodes = {}

    def add_edge(self, v, w, length):
        self.dis_matrix[v][w] = length
        self.dis_matrix[w][v] = length

    def add_node(self, node_idx, node):
        self.nodes[node_idx] = node

    def get_node(self, node_idx):
        return self.nodes[node_idx]

    def adj(self, v):
        return self.matirx[v]


def preprocess_gdf(gdf, graph=Graph()):
    points_dict = {}
    node_index = 0

    for idx, row in gdf.iterrows():
        multilines = row['geometry']
        for points in multilines.geoms:
            xy = points.xy
            if len(xy[0]) >= 2:  # simplify the multilines to lines
                x1, y1 = xy[0][0], xy[1][0]
                x2, y2 = xy[0][-1], xy[1][-1]

                v = points_dict.get((x1, y1))
                if v is None:
                    v = node_index
                    node = Node(x1, y1)
                    graph.add_node(v, node)
                    points_dict[(x1, y1)] = node_index
                    node_index += 1

                w = points_dict.get((x2, y2))
                if w is None:
                    w = node_index
                    node = Node(x2, y2)
                    graph.add_node(w, node)
                    points_dict[(x2, y2)] = w
                    node_index += 1

                # add length
                length = row['SHAPE_Length']
                graph.add_edge(v, w, length)

    return graph

# Graph implemented by edge list::
class GraphFromEdgeList:
    def __init__(self):
        self.edges = []
        self.nodes = set()

    def add_edge(self, source, target, length):
        self.edges.append((source, target, length))
        self.nodes.add(source)
        self.nodes.add(target)

def preprocess_gdf_to_edge_list(gdf):
    edge_list_graph = GraphFromEdgeList()

    for idx, row in gdf.iterrows():
        multilines = row['geometry']
        for points in multilines.geoms:
            xy = points.xy
            if len(xy[0]) >= 2:  # simplify the multilines to lines
                x1, y1 = xy[0][0], xy[1][0]
                x2, y2 = xy[0][-1], xy[1][-1]

                edge_list_graph.add_edge((x1, y1), (x2, y2), row['SHAPE_Length'])

    return edge_list_graph


In [7]:
if __name__ == "__main__":
    gdf = gpd.read_file('Roads_small.gpkg')
    m=0

    '''
    #generate the graph stored in the adjacency list
    road_graph = preprocess_gdf(gdf)
    for v, w in road_graph.adj.items():
        print("Node:", v, "Edges:", w)
        node = road_graph.get_node(v)
        print("Latitude:", node.x, "Longitude:", node.y)
        m+=1
    print(m)

    edge_list_graph = preprocess_gdf_to_edge_list(gdf)
    for edge in edge_list_graph.edges[:10]:
        print("Edge:", edge)
    '''

    matrix_graph = preprocess_gdf(gdf, graph=Graph_matrix())
    print(matrix_graph.dis_matrix)
    print(matrix_graph.nodes[0].pos)

[[  0.         287.80521198   0.         ...   0.           0.
    0.        ]
 [287.80521198   0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]]
(2635432.1275000013, 1170007.039999999)


I will copy the other document here (Graph_adjacency_list.ipynb) so that i can adjust stuff without altering the original.

In [5]:
import json
import geopandas as gpd
import numpy as np
from shapely.geometry import MultiLineString, Point, LineString
import math

# use geopandas to read in the GeoJSON file
roads = gpd.read_file("Roads_small.geojson")

# create a new column that contains the length of each road
viz_roads = roads.copy()
print(viz_roads.head())

#check the data type of the geometry column
print(viz_roads['geometry'].dtype)
print(viz_roads.type)

class Graph:
    def __init__(self):
        self.adjacency_list = {}
        self.edge_lengths = {}

    def add_edge(self, v, w):
        if v not in self.adjacency_list:
            self.adjacency_list[v] = []
        if w not in self.adjacency_list:
            self.adjacency_list[w] = []
        if w not in self.adjacency_list[v] and v not in self.adjacency_list[w]:
            self.adjacency_list[v].append(w)
            self.adjacency_list[w].append(v)
            distance = self.calculate_distance(v, w)
            self.edge_lengths[(v, w)] = distance
            self.edge_lengths[(w, v)] = distance  

    def calculate_distance(self, start, end):
        return math.sqrt((start[0] - end[0]) ** 2 + (start[1] - end[1]) ** 2)

    def get_adjacent(self, v):
        return self.adjacency_list.get(v, [])

    def get_edge_length(self, v, w):
        return self.edge_lengths.get((v, w), None)
        
    def dijkstra(self, source):
        distance = {node: float('inf') for node in self.adjacency_list}
        distance[source] = 0
        visited = set()
        pq = [(0, source)]

        while pq:
            dist_u, u = min(pq)
            pq.remove((dist_u, u))
            if u in visited:
                continue
            visited.add(u)
            for v in self.adjacency_list[u]:
                alt = distance[u] + self.get_edge_length(u, v)
                if alt < distance[v]:
                    distance[v] = alt
                    pq.append((alt, v))

        return distance

    def __str__(self):
        return "Graph with nodes: " + str(list(self.adjacency_list.keys()))

    def display_adjacency_list(self):
        return self.adjacency_list

    def display_edge_lengths(self):
        return self.edge_lengths


   fid  OBJECTID                                DKM_UUID  OBJEKTART  NAME  \
0   95      2126  {84F4C50E-E009-4DB8-AF3C-DEE20F3177F6}       1700  None   
1  113      2145  {6408AD08-C7B5-4602-81D7-A9B7FBFFD5FF}       1700  None   
2  114      2146  {578A6D84-CC7B-4687-B7C6-79313525392A}       1700  None   
3  163      2195  {BF0A1AEF-817C-45DA-A33C-6C37903D19BB}       1500  None   
4  169      2201  {C3EB4DD7-7E8F-4BB5-9DFD-BEE78567F8A7}       1400  None   

  ROUTENNUMMER   SG  STUFE  KUNSTBAUTE  BEFAHRBARKEIT  ...  OVERRIDE1 OFF2  \
0         None  200      0         100            -99  ...       None  NaN   
1         None  200      0         100            -99  ...       None  NaN   
2         None  200      0         100            -99  ...       None  NaN   
3         None  200      0         100              1  ...       None  NaN   
4         None  200      0         100              1  ...       None  NaN   

   OFF3  CUT_BEGIN_FB  CUT_END_FB  EXTREMITY_TYPE2  NAME_LANG  INVER

In [14]:
# Create a graph
graph = Graph()

# Example DataFrame processing loop
for index, row in viz_roads.iterrows():  
    geometry = row['geometry']
    if isinstance(geometry, MultiLineString):
        for line in geometry.geoms:
            start_point = (line.coords[0][0], line.coords[0][1])
            end_point = (line.coords[-1][0], line.coords[-1][1])
            graph.add_edge(start_point, end_point)
    else:  # Single LineString
        start_point = (geometry.coords[0][0], geometry.coords[0][1])
        end_point = (geometry.coords[-1][0], geometry.coords[-1][1])
        graph.add_edge(start_point, end_point)

# Print the first 10 rows of the adjacency list
print("Adjacency List:")
for index, (key, value) in enumerate(graph.display_adjacency_list().items()):
    if index >= 10:
        break
    print(f"{key}: {value}")

# Print the first 10 rows of the edge lengths
print("Edge Lengths:")
for index, (key, length) in enumerate(graph.display_edge_lengths().items()):
    if index >= 10:
        break
    print(f"{key}: {length:.2f} units")


graph.dijkstra((2635432.1275000013, 1170007.039999999))

Adjacency List:
(2635432.1275000013, 1170007.039999999): [(2635453.1999999993, 1169889.1999999993), (2635524.2025000006, 1169944.8099999987)]
(2635453.1999999993, 1169889.1999999993): [(2635432.1275000013, 1170007.039999999), (2635547.719999999, 1169917.8237500004), (2635351.6712500006, 1169739.4262499996)]
(2635401.647500001, 1170101.65625): [(2635524.2025000006, 1169944.8099999987)]
(2635524.2025000006, 1169944.8099999987): [(2635401.647500001, 1170101.65625), (2635432.1275000013, 1170007.039999999), (2635547.719999999, 1169917.8237500004)]
(2635992.8337499984, 1170210.0287499987): [(2635883.401250001, 1169916.4475000016), (2636567.0862499997, 1170509.75), (2635862.446249999, 1170227.8099999987), (2636316.326250002, 1170622.7025000006)]
(2635883.401250001, 1169916.4475000016): [(2635992.8337499984, 1170210.0287499987), (2635775.8999999985, 1169816.6999999993)]
(2635582.1999999993, 1170101.4437500015): [(2635566.513750002, 1169921.7250000015), (2635424.9312499985, 1170175.7387499996),

{(2635432.1275000013, 1170007.039999999): 0,
 (2635453.1999999993, 1169889.1999999993): 119.70929728357342,
 (2635401.647500001, 1170101.65625): 310.18118095289384,
 (2635524.2025000006, 1169944.8099999987): 111.13225690553772,
 (2635992.8337499984, 1170210.0287499987): 864.0598533012974,
 (2635883.401250001, 1169916.4475000016): 550.7462366079344,
 (2635582.1999999993, 1170101.4437500015): 346.5243451786315,
 (2635566.513750002, 1169921.7250000015): 166.1223287249745,
 (2650418.0512499996, 1169875.1675000004): 17974.87734148533,
 (2649934.8150000013, 1170260.4849999994): 17356.826494776276,
 (2646198.625, 1169563.004999999): 12848.927593935903,
 (2647121.879999999, 1170009.4987500012): 13874.479378983413,
 (2637659.5, 1170215.9562500007): 2910.019890880291,
 (2637429.2062500007, 1170638.4437500015): 2767.1113897862565,
 (2646029.1537499987, 1170318.8062499985): 12878.271414239243,
 (2645542.4137500003, 1169684.8625000007): 12079.021449735252,
 (2638116.2962500006, 1169981.1000000015):

Apparently this Algorithm works. I've got to revise it further and also implement an end node

Not entirely sure if it worked correctly, but hey, I am just going to write a dijkstra algorithm.

In [ ]:
# some pseudocode here, without actually trying if something works

# Initial length list
# All nodes inf except the one chosen as starting point
node_list = {} # Store all nodes in graph here
value = 'inf' # Initial value for all nodes
path_dict = dict.fromkeys(node_list, value) 

start_node = X # To be decided on
path_dict.update({startnode: 0})

# choose node with min distance (assuming there is only one node with that value)
next_node = min(node_list.values())



 # Let's start new 

 I am going to try and implement a Dijkstra again based of the updated graph.py file:
 

In [4]:
import geopandas as gpd
import numpy as np
from geospatial import *
from graph import *

Would be working, but since I need to change things in the code, I will copy it here to not mess up the original code: 

In [40]:
class Graph:
    def __init__(self):
        self.adj_list = {}
        self.nodes = {}

    def add_edge(self, v, w, length):
        self.adj_list.setdefault(v, []).append((w, length))
        self.adj_list.setdefault(w, []).append((v, length))

    def add_node(self, node_idx, node):
        self.nodes[node_idx] = node

    def get_node(self, node_idx):
        return self.nodes[node_idx]
    
    def get_edge_length(self, v, w):
        for node, length in self.adj_list[v]:
            if node == w:
                return length
        return None

    def adj(self, v):
        return self.adj[v]
    
    def dijkstra(graph, start_node):
        distance = {node: float('inf') for node in graph.nodes}
        distance[start_node] = 0
        visited = set()
        pq = [(0, start_node)]

        while pq: 
            dist_u, u = min(pq)
            pq.remove((dist_u, u))
            if u in visited:
                continue
            visited.add(u)
            for v in graph.adj_list[u]:
                alt = distance[u] + graph.get_edge_length(u,v)
                if alt < distance[v]:
                     distance[v] = alt
                     pq.append((alt,v))

        return distance
    
    def dijkstra(graph, start_node):
        distance = {node: float('inf') for node in graph.nodes}
        distance[start_node] = 0
        visited = set()
        pq = [(0, start_node)]

        while pq: 
            dist_u, u = min(pq)
            pq.remove((dist_u, u))
            if u in visited:
                continue
            visited.add(u)
            if u in graph.adj_list:
                for v, _ in graph.adj_list[u]:
                    alt = distance[u] + graph.get_edge_length(u, v)
                    if alt < distance.get(v, float('inf')):
                        distance[v] = alt
                        pq.append((alt, v))          

        return distance

    def dijkstra_with_end_node(graph, start_node, end_node):
        distance = {node: float('inf') for node in graph.nodes}
        distance[start_node] = 0
        visited = set()
        pq = [(0, start_node)]    

        while pq: 
            dist_u, u = min(pq)
            pq.remove((dist_u, u))
            if u in visited:
                continue
            visited.add(u)
            if u in graph.adj_list:
                for v, _ in graph.adj_list[u]:
                    alt = distance[u] + graph.get_edge_length(u, v)
                    if alt < distance.get(v, float('inf')):
                        distance[v] = alt
                        pq.append((alt, v))          

        return distance[end_node]           



In [41]:
graph = Graph()
'''
graph.add_node(1, 'A')
graph.add_node(2, 'B')
graph.add_node(3, 'C')
graph.add_node(4, 'D')
graph.add_node(5, 'E')
'''
graph.add_edge('A', 'B', 4)
graph.add_edge('A', 'C', 2)
graph.add_edge('B', 'C', 5)
graph.add_edge('B', 'D', 10)
graph.add_edge('C', 'D', 3)
graph.add_edge('E', 'A', 7)

start_node = 'A'
end_node = 'D'
distance= graph.dijkstra_with_end_node(start_node, end_node)
print("The shortest distance from node", start_node, "to node", end_node, "is:", distance)


The shortest distance from node A to node D is: 5


So that was the adjacency list. Let's continue with the matrix

In [42]:
# Graph implemented by adjacency matrix
class Graph_matrix:
    def __init__(self):
        self.dis_matrix= np.zeros((4965, 4956))
        self.nodes = {}

    def add_edge(self, v, w, length):
        self.dis_matrix[v][w] = length
        self.dis_matrix[w][v] = length

    def add_node(self, node_idx, node):
        self.nodes[node_idx] = node

    def get_node(self, node_idx):
        return self.nodes[node_idx]

    def adj(self, v):
        return self.matirx[v]

